In [10]:
import requests
import pandas as pd

# API setup
base_url_fwd = "https://fiat.api.newchangefx.com/historic/fwdcurve/v1/rates"
timestamp_fwd = "2024-07-18"
headers = {
    'x-api-key': "97F56251C827E508DFBC02507BCA250E139A1C04490CFEA50913D544CE074B81"
}

# Function to call the API and get the forward curve
def get_forward_curve(base_url, timestamp, headers):
    params = {
        'currencyPair': 'EURUSD',
        'timestamp': timestamp
    }
    try:
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Function to calculate straight-line values between tenors
def calculate_straightline(df):
    df['m'] = df['m'].astype(float)
    df['vd'] = pd.to_datetime(df['vd'])
    df['ppd'] = df['ppd'].astype(float).fillna(0)
    tenors = df[df['tn'].notnull() & ~df['tn'].isin(['ON', 'TN'])]

    # Create a new DataFrame to hold the straight-line values
    straightline_data = []

    # Iterate through the tenors to interpolate between them
    for i in range(len(tenors) - 1):
        start_idx = tenors.index[i]
        end_idx = tenors.index[i + 1]
        start_date = df.at[start_idx, 'vd']
        end_date = df.at[end_idx, 'vd']
        start_value = df.at[start_idx, 'm']
        end_value = df.at[end_idx, 'm']
        num_days = (end_date - start_date).days

        # Calculate daily increment
        daily_increment = (end_value - start_value) / num_days

        for j in range(num_days):
            current_date = start_date + pd.Timedelta(days=j)
            current_value = start_value + daily_increment * j
            straightline_data.append({
                'Value Date': current_date,
                'Forwards365 Value': df.at[start_idx + j, 'm'],
                'SL Value': current_value,
                'Fwd365 points per day': df.at[start_idx + j, 'ppd'],
                'SL points per day': daily_increment
            })

    straightline_df = pd.DataFrame(straightline_data)
    return straightline_df

# Get the forward curve data
forward_curve_data = get_forward_curve(base_url_fwd, timestamp_fwd, headers)

if forward_curve_data:
    # Extract relevant data from JSON
    data = forward_curve_data['rates']
    df = pd.DataFrame(data)

    # Calculate straight-line values between tenors
    df = calculate_straightline(df)

    # Select and rename columns for output
    df = df[['Value Date', 'Forwards365 Value', 'SL Value', 'Fwd365 points per day', 'SL points per day']]
    
    # Convert to CSV
    output_file = r"C:\Users\XavierPorterfield\OneDrive - New Change FX\Data- NCFX\Forwards\Curves\forward_curve_with_straightline.csv"
    df.to_csv(output_file, index=False)

    print(f"Data exported to {output_file}")
else:
    print("Failed to retrieve forward curve data.")


Data exported to C:\Users\XavierPorterfield\OneDrive - New Change FX\Data- NCFX\Forwards\Curves\forward_curve_with_straightline.csv


In [12]:
import requests
import pandas as pd

# API setup
base_url_fwd = "https://fiat.api.newchangefx.com/historic/fwdcurve/v1/rates"
timestamp_fwd = "2024-07-18 16:00:00"
headers = {
    'x-api-key': "97F56251C827E508DFBC02507BCA250E139A1C04490CFEA50913D544CE074B81"
}

base_url_spot = "https://fiat.api.newchangefx.com/historic/midrate/v1/rates"
timestamp_spot = "2024-07-18 16:00:00.000"

# Function to call the API and get the forward curve
def get_forward_curve(base_url, timestamp, headers):
    params = {
        'currencyPair': 'EURUSD',
        'timestamp': timestamp
    }
    try:
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Function to call the API and get the spot rate
def get_spot_rate(base_url, timestamp, headers):
    params = {
        'currencyPair': 'EURUSD',
        'timestamp': timestamp,
        'tenor': 'SPT'
    }
    try:
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Function to calculate straight-line values between tenors
def calculate_straightline(df):
    df['m'] = df['m'].astype(float)
    df['vd'] = pd.to_datetime(df['vd'])
    df['ppd'] = df['ppd'].astype(float).fillna(0)
    tenors = df[df['tn'].notnull() & ~df['tn'].isin(['ON', 'TN'])]

    # Create a new DataFrame to hold the straight-line values
    straightline_data = []

    # Iterate through the tenors to interpolate between them
    for i in range(len(tenors) - 1):
        start_idx = tenors.index[i]
        end_idx = tenors.index[i + 1]
        start_date = df.at[start_idx, 'vd']
        end_date = df.at[end_idx, 'vd']
        start_value = df.at[start_idx, 'm']
        end_value = df.at[end_idx, 'm']
        num_days = (end_date - start_date).days

        # Calculate daily increment
        daily_increment = (end_value - start_value) / num_days

        for j in range(num_days):
            current_date = start_date + pd.Timedelta(days=j)
            current_value = start_value + daily_increment * j
            straightline_data.append({
                'Value Date': current_date,
                'Forwards365 Value': df.at[start_idx + j, 'm'],
                'SL Value': current_value,
                'Fwd365 points per day': df.at[start_idx + j, 'ppd'],
                'SL points per day': daily_increment
            })

    straightline_df = pd.DataFrame(straightline_data)
    return straightline_df

# Get the forward curve data
forward_curve_data = get_forward_curve(base_url_fwd, timestamp_fwd, headers)

if forward_curve_data:
    # Extract relevant data from JSON
    data = forward_curve_data['rates']
    df = pd.DataFrame(data)

    # Calculate straight-line values between tenors
    df = calculate_straightline(df)

    # Get the spot rate data
    spot_rate_data = get_spot_rate(base_url_spot, timestamp_spot, headers)

    if spot_rate_data:
        spot_rate = float(spot_rate_data['m'])
        
        # Calculate the outright values
        df['Outright Fwd365'] = df['Forwards365 Value'] + spot_rate
        df['Outright SL'] = df['SL Value'] + spot_rate

        # Select and rename columns for output
        df = df[['Value Date', 'Forwards365 Value', 'SL Value', 'Fwd365 points per day', 'SL points per day', 'Outright Fwd365', 'Outright SL']]
        
        # Convert to CSV
        output_file = r"C:\Users\XavierPorterfield\OneDrive - New Change FX\Data- NCFX\Forwards\Curves\forward_curve_with_straightline.csv"
        df.to_csv(output_file, index=False)

        print(f"Data exported to {output_file}")
    else:
        print("Failed to retrieve spot rate data.")
else:
    print("Failed to retrieve forward curve data.")


Data exported to C:\Users\XavierPorterfield\OneDrive - New Change FX\Data- NCFX\Forwards\Curves\forward_curve_with_straightline.csv


In [13]:
import requests
import pandas as pd

# API setup
base_url_fwd = "https://fiat.api.newchangefx.com/historic/fwdcurve/v1/rates"
timestamp_fwd = "2024-07-18 16:00:00"
headers = {
    'x-api-key': "97F56251C827E508DFBC02507BCA250E139A1C04490CFEA50913D544CE074B81"
}

base_url_spot = "https://fiat.api.newchangefx.com/historic/midrate/v1/rates"
timestamp_spot = "2024-07-18 16:00:00.000"

# Conversion factor for forward points to true decimal
conversion_factor = 10000

# Function to call the API and get the forward curve
def get_forward_curve(base_url, timestamp, headers):
    params = {
        'currencyPair': 'EURUSD',
        'timestamp': timestamp
    }
    try:
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Function to call the API and get the spot rate
def get_spot_rate(base_url, timestamp, headers):
    params = {
        'currencyPair': 'EURUSD',
        'timestamp': timestamp,
        'tenor': 'SPT'
    }
    try:
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Function to calculate straight-line values between tenors
def calculate_straightline(df):
    df['m'] = df['m'].astype(float)
    df['vd'] = pd.to_datetime(df['vd'])
    df['ppd'] = df['ppd'].astype(float).fillna(0)
    tenors = df[df['tn'].notnull() & ~df['tn'].isin(['ON', 'TN'])]

    # Create a new DataFrame to hold the straight-line values
    straightline_data = []

    # Iterate through the tenors to interpolate between them
    for i in range(len(tenors) - 1):
        start_idx = tenors.index[i]
        end_idx = tenors.index[i + 1]
        start_date = df.at[start_idx, 'vd']
        end_date = df.at[end_idx, 'vd']
        start_value = df.at[start_idx, 'm']
        end_value = df.at[end_idx, 'm']
        num_days = (end_date - start_date).days

        # Calculate daily increment
        daily_increment = (end_value - start_value) / num_days

        for j in range(num_days):
            current_date = start_date + pd.Timedelta(days=j)
            current_value = start_value + daily_increment * j
            straightline_data.append({
                'Value Date': current_date,
                'Forwards365 Value': df.at[start_idx + j, 'm'],
                'SL Value': current_value,
                'Fwd365 points per day': df.at[start_idx + j, 'ppd'],
                'SL points per day': daily_increment
            })

    straightline_df = pd.DataFrame(straightline_data)
    return straightline_df

# Get the forward curve data
forward_curve_data = get_forward_curve(base_url_fwd, timestamp_fwd, headers)

if forward_curve_data:
    # Extract relevant data from JSON
    data = forward_curve_data['rates']
    df = pd.DataFrame(data)

    # Calculate straight-line values between tenors
    df = calculate_straightline(df)

    # Get the spot rate data
    spot_rate_data = get_spot_rate(base_url_spot, timestamp_spot, headers)

    if spot_rate_data:
        spot_rate = float(spot_rate_data['m'])
        
        # Convert forward values to true decimal
        df['Forwards365 Value (decimal)'] = df['Forwards365 Value'] / conversion_factor
        df['SL Value (decimal)'] = df['SL Value'] / conversion_factor

        # Calculate the outright values
        df['Outright Fwd365'] = df['Forwards365 Value (decimal)'] + spot_rate
        df['Outright SL'] = df['SL Value (decimal)'] + spot_rate

        # Select and rename columns for output
        df = df[['Value Date', 'Forwards365 Value', 'SL Value', 'Fwd365 points per day', 'SL points per day', 'Outright Fwd365', 'Outright SL']]
        
        # Convert to CSV
        output_file = r"C:\Users\XavierPorterfield\OneDrive - New Change FX\Data- NCFX\Forwards\Curves\forward_curve_with_straightlineOutright.csv"
        df.to_csv(output_file, index=False)

        print(f"Data exported to {output_file}")
    else:
        print("Failed to retrieve spot rate data.")
else:
    print("Failed to retrieve forward curve data.")

Data exported to C:\Users\XavierPorterfield\OneDrive - New Change FX\Data- NCFX\Forwards\Curves\forward_curve_with_straightlineOutright.csv


In [14]:
import requests
import pandas as pd

# API setup
base_url_fwd = "https://fiat.api.newchangefx.com/historic/fwdcurve/v1/rates"
timestamp_fwd = "2024-07-18 16:00:00"
headers = {
    'x-api-key': "97F56251C827E508DFBC02507BCA250E139A1C04490CFEA50913D544CE074B81"
}

base_url_spot = "https://fiat.api.newchangefx.com/historic/midrate/v1/rates"
timestamp_spot = "2024-07-18 16:00:00.000"

# Conversion factor for forward points to true decimal
conversion_factor = 10000

# Notional trade amount in EUR
notional_trade_amount = 1000000

# Function to call the API and get the forward curve
def get_forward_curve(base_url, timestamp, headers):
    params = {
        'currencyPair': 'EURUSD',
        'timestamp': timestamp
    }
    try:
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Function to call the API and get the spot rate
def get_spot_rate(base_url, timestamp, headers):
    params = {
        'currencyPair': 'EURUSD',
        'timestamp': timestamp,
        'tenor': 'SPT'
    }
    try:
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Function to calculate straight-line values between tenors
def calculate_straightline(df):
    df['m'] = df['m'].astype(float)
    df['vd'] = pd.to_datetime(df['vd'])
    df['ppd'] = df['ppd'].astype(float).fillna(0)
    tenors = df[df['tn'].notnull() & ~df['tn'].isin(['ON', 'TN'])]

    # Create a new DataFrame to hold the straight-line values
    straightline_data = []

    # Iterate through the tenors to interpolate between them
    for i in range(len(tenors) - 1):
        start_idx = tenors.index[i]
        end_idx = tenors.index[i + 1]
        start_date = df.at[start_idx, 'vd']
        end_date = df.at[end_idx, 'vd']
        start_value = df.at[start_idx, 'm']
        end_value = df.at[end_idx, 'm']
        num_days = (end_date - start_date).days

        # Calculate daily increment
        daily_increment = (end_value - start_value) / num_days

        for j in range(num_days):
            current_date = start_date + pd.Timedelta(days=j)
            current_value = start_value + daily_increment * j
            straightline_data.append({
                'Value Date': current_date,
                'Forwards365 Value': df.at[start_idx + j, 'm'],
                'SL Value': current_value,
                'Fwd365 points per day': df.at[start_idx + j, 'ppd'],
                'SL points per day': daily_increment
            })

    straightline_df = pd.DataFrame(straightline_data)
    return straightline_df

# Get the forward curve data
forward_curve_data = get_forward_curve(base_url_fwd, timestamp_fwd, headers)

if forward_curve_data:
    # Extract relevant data from JSON
    data = forward_curve_data['rates']
    df = pd.DataFrame(data)

    # Calculate straight-line values between tenors
    df = calculate_straightline(df)

    # Get the spot rate data
    spot_rate_data = get_spot_rate(base_url_spot, timestamp_spot, headers)

    if spot_rate_data:
        spot_rate = float(spot_rate_data['m'])
        
        # Convert forward values to true decimal
        df['Forwards365 Value (decimal)'] = df['Forwards365 Value'] / conversion_factor
        df['SL Value (decimal)'] = df['SL Value'] / conversion_factor

        # Calculate the outright values
        df['Outright Fwd365'] = df['Forwards365 Value (decimal)'] + spot_rate
        df['Outright SL'] = df['SL Value (decimal)'] + spot_rate

        # Multiply by notional trade amount
        df['Outright Fwd365 Notional'] = df['Outright Fwd365'] * notional_trade_amount
        df['Outright SL Notional'] = df['Outright SL'] * notional_trade_amount

        # Calculate the difference in USD terms
        df['Difference in USD'] = df['Outright SL Notional'] - df['Outright Fwd365 Notional']

        # Select and rename columns for output
        df = df[['Value Date', 'Forwards365 Value', 'SL Value', 'Fwd365 points per day', 'SL points per day', 'Outright Fwd365', 'Outright SL', 'Outright Fwd365 Notional', 'Outright SL Notional', 'Difference in USD']]
        
        # Convert to CSV
        output_file = r"C:\Users\XavierPorterfield\OneDrive - New Change FX\Data- NCFX\Forwards\Curves\forward_curve_with_straightline.csv"
        df.to_csv(output_file, index=False)

        print(f"Data exported to {output_file}")
    else:
        print("Failed to retrieve spot rate data.")
else:
    print("Failed to retrieve forward curve data.")


Data exported to C:\Users\XavierPorterfield\OneDrive - New Change FX\Data- NCFX\Forwards\Curves\forward_curve_with_straightline.csv
